In [1]:
import json

In [2]:
import time

In [3]:
from tqdm import tqdm

In [4]:
with open('grade_school_math/data/test.jsonl', 'r') as file_split:
    json_list = list(file_split)

split = []
for json_str in json_list:
    result = json.loads(json_str)
    split.append(result)

In [5]:
len(split)

1319

In [6]:
split[0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [5]:
import re
import ast

In [6]:
import amrlib
import spacy
amrlib.setup_spacy_extension()
nlp = spacy.load('en_core_web_sm')

/data/wang/zhenyub/condaenvs/cenv_x86/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [7]:
from openai import OpenAI
client = OpenAI(api_key='YOUR_KEY')

# Set up YOUR OpenAI API key
#OPENAI_API_KEY = "YOUR_KEY"
# Set MODEL to "gpt-3.5-turbo"
model = "gpt-3.5-turbo"
# Set temperature to 0.9, it can be from 0~1, 0 is the most conservative, 1 is the most creative
temperature = 0.5
messages = [
    {"role": "system", "content": "You are a helpful and honest assistant. \
      Respond concisely and truthfully."},
    {"role": "user", "content": "Explain Einstein's relativity theory in three sentences."},
]

# Send the message to the bot
response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature)

# Get the response from the bot
response = response_message.choices[0].message.content

print(response)

Einstein's theory of relativity, comprised of special and general relativity, revolutionized our understanding of space, time, and gravity. Special relativity states that the laws of physics are the same for all observers in uniform motion, while general relativity describes gravity as the curvature of spacetime caused by mass and energy. Together, these theories have profound implications for our understanding of the universe and have been confirmed through various experiments and observations.


In [10]:
def Answer_Agent_IO(question='', num_seq=1):
    message = \
    f"""Given a question, give out only the direct answer, without any explanations. \
    Here are two examples as guideline: \
    "Question: John bought 3 cars, each worth $30000. How much did John pay in total for his 3 cars? \
    Answer: ###$9000###. \
    Question: Payton has 9 apples that she is planning to evenly split to 3 of her friends. How many apples would each of her friend get? \
    Answer: ###3###." \
    Give out the final answer in numerical format. \
    Make sure to include your answer in the format: ###answer###. \
    Question: {question} \
    Answer: \
    """
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer mathematical questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature)

    response = response_message.choices[0].message.content

    return response

In [8]:
def Answer_Agent_CoT_2Shot(question='', num_seq=1):
    message = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Here are two examples as guideline: \
    "Question: John bought 3 cars, each worth $30000. He also bought 3 bikes, each worth $1000. How much did John pay in total? \
    Thought: John paid 3 * $30000 = $90000 for the cars, and 3 *$ 1000 = $3000 for the bikes. The total amounts to $90000 + $3000 = $93000. \
    Answer: ###$93000###. \
    Question: Each day, Payton brings 9 apples that she would split to 3 of her friends. In 3 days, how many apples would each of her friend get in total? \
    Thought: Each of the 3 friends would get 9 / 3 = 3 apples per day. For 3 days, each of her friend would get 3 * 3 = 9 apples in total. \
    Answer: ###9###." \
    Give out the answer in numerical format. \
    Make sure to include your final answer in the format: ###answer###. \
    Question: {question} \
    Answer: \
    """
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer mathematical questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature, n=num_seq)

    response = [response_message.choices[i].message.content for i in range(num_seq)]
    # response = response_message.choices[0].message.content

    return response

In [9]:
def Answer_Agent_CoT_0Shot(question='', num_seq=1):
    message = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Make sure to include your final answer to the question in the format: ###your final answer###. \
    Give out the answer in numerical format. \
    Question: {question} \
    Let's think step by step. \
    Answer: \
    """
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer mathematical questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature)

    response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature, n=num_seq)

    response = [response_message.choices[i].message.content for i in range(num_seq)]
    # response = response_message.choices[0].message.content

    return response

In [22]:
s = '###123###'
ast.literal_eval(re.search('###(.*)###', s).group(1))

123

In [16]:
all_answer_lm3_text_io = []
ori_answers_io = []
all_counts = -1
for s in tqdm(split[0:100]):
    all_counts += 1
    ori_question = s['question']
    
    response = Answer_Agent_IO(ori_question, 1)
    ori_answers_io.append(response)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_io.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_io.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_io.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_io.append('')
                continue

100%|██████████| 100/100 [00:28<00:00,  3.57it/s]


In [70]:
all_answer_lm3_text_cot = []
ori_reasoning_cot = []
all_counts = -1
for s in tqdm(split[0:100]):
    all_counts += 1
    ori_question = s['question']
    
    response = Answer_Agent_CoT_2Shot(ori_question, 1)[0]
    ori_reasoning_cot.append(response)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_cot.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_cot.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_cot.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_cot.append('')
                continue

100%|██████████| 100/100 [02:20<00:00,  1.41s/it]


In [82]:
all_answer_lm3_text_cot_0s = []
ori_reasoning_cot_0s = []
all_counts = -1
for s in tqdm(split[0:100]):
    all_counts += 1
    ori_question = s['question']
    
    response = Answer_Agent_CoT_0Shot(ori_question, 3)[0]
    ori_reasoning_cot_0s.append(response)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_cot_0s.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_cot_0s.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_cot_0s.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_cot_0s.append('')
                continue

  4%|▍         | 4/100 [00:13<05:09,  3.23s/it]

no answer provided
Step 1: Calculate the total distance James runs in one sprint: 60 meters/sprint
Step 2: Calculate the total distance James runs in one week: 60 meters/sprint * 3 sprints * 3 times a week
Step 3: Multiply the total distance in one week by the number of weeks: total distance in one week * 1 week

Final Answer: 540 meters.


 11%|█         | 11/100 [00:46<06:13,  4.20s/it]


KeyboardInterrupt: 

In [51]:
all_answer_lm3_text_io_ensemble3 = []
all_counts = -1
for s in tqdm(split[0:100]):
    all_counts += 1
    ori_question = s['question']
    
    responses = Answer_Agent_IO(ori_question, 3)
    answer_current = []

    for response in responses:
        try:
            response_format = re.search('###(.*)###', response).group(1)
            try:
                answer_current.append(ast.literal_eval(response_format))
            except:
                answer_current.append(response_format)
        except:
            try:
                answer_current.append(ast.literal_eval(response))
            except:
                try:
                    response_format = response.split('Answer: ')[1]
                    answer_current.append(ast.literal_eval(response_format))
                except:
                    print('no answer provided')
                    print(response)
                    answer_current.append('')
                    continue
    all_answer_lm3_text_io_ensemble3.append(answer_current)

100%|██████████| 100/100 [04:49<00:00,  2.89s/it]


In [52]:
all_answer_lm3_text_io_ensemble3

[['$38', '$34', '$34'],
 [4, 4, 4],
 ['$106,000', '$98,000', '$110,000'],
 [1080, 1080, 1080],
 [65, 65, 75],
 ['$80', '$96', '$80'],
 [180, 180, 140],
 ['(200 * 0.4 * 2) + (200 * 0.6 * 2) + 20', 200, 200],
 ['distance = 0',
  '###distance = 0###',
  'John drives 3 hours at 60 mph, so he covers 3 * 60 = 180 miles from home initially. Then, he drives 0.5 hours at 30 mph, so he covers 0.5 * 30 = 15 miles. The remaining distance to home is (4 - 0.5 - 2) * 80 = 1.6 * 80 = 128 miles. So the total distance is 180 + 15 + 128 = 323 miles.'],
 ['$460', '$460', '$450 + ($10 * 5 * 1.2)'],
 [180, 240, 180],
 ['3 * $68 = $204', '3 * $68 = $204', '3 * $68 = $204'],
 [7, 7, 7],
 [12, 12, 12],
 ['60%', '60%', '60%'],
 ['$150', '$500', '$-500'],
 ["Since the question doesn't specify the distance covered by each train, we'll assume they cover the same distance. So, each train would cover 80 + 150 = 230 miles in the two days.",
  'Since the trains travel the same distance, we can find the total distance 

In [12]:
import sys
import re
import string
from collections import Counter
import pickle

In [13]:
def normalize_answer(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [17]:
with open('answers_gpt3_io.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_io))

In [73]:
with open('answers_gpt3_io_ori.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in ori_answers_io))

In [71]:
with open('answers_gpt3_cot_0Shot_ori.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in ori_reasoning_cot_0s))

In [70]:
with open('answers_gpt3_cot_0Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot_0s))

In [75]:
with open('answers_gpt3_cot_2Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot))

In [43]:
with open('answers_golden.txt', 'w') as f:
    f.write('\n'.join(s['answer'].split('#### ')[1] for s in split[0:100]))

In [14]:
ANS_RE = re.compile(r"(\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"

def extract_answer(completion):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        if len(match_str) == 0:
            return INVALID_ANS
        if match_str[-1] == '.':
            match_str = match_str[:-1]
        if len(match_str) == 0:
            return INVALID_ANS
        return match_str
    else:
        return INVALID_ANS

In [15]:
def is_correct(model_completion, gt_example):
    gt_answer = extract_answer(gt_example)
    assert gt_answer != INVALID_ANS
    model_answer = extract_answer(model_completion)
    if model_answer == INVALID_ANS:
        return False
    return int(float(model_answer)) == int(float(gt_answer))
    # final_ans = extract_answer(model_completion) == gt_answer
    # return final_ans

In [39]:
with open('answers_golden.txt', 'r') as file:
    golden = file.readlines()

In [21]:
with open('answers_gpt3_io.txt', 'r') as file:
    output_io = file.readlines()

with open('answers_gpt3_cot_0Shot.txt', 'r') as file:
    output_cot_0Shot = file.readlines()

with open('answers_gpt3_cot_2Shot.txt', 'r') as file:
    output_cot_2Shot = file.readlines()

In [41]:
def evaluation(output, golden):
    correct = 0
    length = len(golden)
    list_wrong = []
    i = 0

    for (o, g) in zip(output, golden):
        res = is_correct(o, g)
        if res:
            correct += 1
        else:
            list_wrong.append(i)
        i += 1
    
    return correct, correct/length, list_wrong

In [23]:
c, a, wrong = evaluation(output_io, golden)
c, a

(35, 0.35)

In [24]:
c, p, l = evaluation(output_cot_0Shot, golden)
c, p

(80, 0.8)

In [25]:
c, p, wrong = evaluation(output_cot_2Shot, golden)
c, p

(80, 0.8)

In [34]:
def Answer_Agent_debate(ori_question='', rounds=1, num_agents=3):
    message_user_prior = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Make sure to include your answer in the format: ###your answer###. \
    Give out the answer in numerical format. \
    Question: {ori_question} \
    Let's think step by step. \
    Answer: \
    """

    message = \
    f"""Given a question, and some previous reasoning, recheck the answers and give out your explanation. \
    If you think your previous answer is wrong, provide the correct answer and your reasoning for it. \
    If you think your previous answer is correct, explain why it is correct. \
    Make sure to include your final answer in the format: ###answer###. \
    Question: {ori_question} \
    """

    messages = [{"role": "system", "content": \
                        "You are a faithful agent that answer mathematical questions accurately."}]
    
    all_responses = {}
    for i in range(num_agents):
        all_responses[i] = {'user': [], 'assistant': []}
    
    response_list = Answer_Agent_CoT_0Shot(ori_question, num_agents)
    for j in range(len(response_list)):
        all_responses[j]['user'].append(message_user_prior)
        all_responses[j]['assistant'].append(response_list[j])
    
    for r in range(rounds):
        for i in range(num_agents):
            new_messages = messages
            #special case for corners r-1
            new_messages.append({"role": "user", "content": all_responses[i]['user'][0]})
            new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][0]})
            
            if r != 0:
                new_messages.append({"role": "user", "content": all_responses[i]['user'][r]})
                new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][r]})
            
            other_responses = response_list[:i] + response_list[i+1:]
            message += f'Reasoning from other agents: '
            for k in range(len(other_responses)):
                message += f'Agent {k+1}: {other_responses[k]}'
            message += 'Thought:'
            new_messages.append({"role": "user", "content": message})
            
            response_message = client.chat.completions.create(model=model, messages=new_messages, temperature=temperature)
            response = response_message.choices[0].message.content
            all_responses[i]['user'].append(message)
            all_responses[i]['assistant'].append(response)

    return all_responses

In [35]:
result = []
for li in l:
    result.append(Answer_Agent_debate(split[li], 2, 3))

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 18520 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [91]:
result

[{0: ['To calculate the profit Josh made from flipping the house, we need to follow these steps:\n\n1. Josh bought the house for $80,000.\n2. He then put in $50,000 in repairs.\n3. The total cost of the house and repairs is $80,000 + $50,000 = $130,000.\n4. The value of the house increased by 150%, which means it increased by 1.5 times.\n5. The increase in value is $80,000 * 1.5 = $120,000.\n6. The new value of the house after repairs is $120,000 + $80,000 = $200,000.\n7. To find the profit, we subtract the total cost from the new value: $200,000 - $130,000 = $70,000.\n\n###The profit Josh made from flipping the house is $70,000###.',
   "The previous agent's chain of thought and the final answer provided are correct.\n\n1. The total cost of the house and repairs is calculated correctly as $130,000 ($80,000 + $50,000).\n2. The increase in value after repairs is correctly calculated as $120,000 ($80,000 * 1.5).\n3. The new value of the house after repairs is correctly determined as $200

In [1]:
central pool
whether 3 agents agree with the pool

SyntaxError: invalid syntax (663562552.py, line 1)

In [36]:
def Answer_Agent_debate(ori_question='', rounds=1, num_agents=3):
    message_user_prior = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Make sure to include your answer in the format: ###your answer###. \
    Give out the answer in numerical format. \
    Question: {ori_question} \
    Let's think step by step. \
    Answer: \
    """

    message = \
    f"""Given some potential answers given by other agents: \
    if you think your previous answer is wrong, and one of the potential answers is correct, \
    provide the correct answer and your reasoning for it; \
    if you think your previous answer is correct, explain why it is correct. \
    Make sure to include your final answer in the format: ###answer###. \
    """
    #    Question: {ori_question} \

    messages = [{"role": "system", "content": \
                        "You are a faithful agent that answer mathematical questions accurately."}]
    
    all_responses = {}
    for i in range(num_agents):
        all_responses[i] = {'user': [], 'assistant': []}
    response_pool = set()
    
    response_list = Answer_Agent_CoT_0Shot(ori_question, num_agents)
    for j in range(len(response_list)):
        all_responses[j]['user'].append(message_user_prior)
        try:
            response_format = re.search('###(.*)###', response_list[j]).group(1)
            all_responses[j]['assistant'].append(response_format)
            extracted = extract_answer(response_format)
            if extracted != INVALID_ANS:
                response_pool.add(extracted)
        except:
            response_format = response_list[j]
    
    for r in range(rounds):
        for i in range(num_agents):
            new_messages = messages
            new_message = message

            new_messages.append({"role": "user", "content": all_responses[i]['user'][0]})
            new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][0]})
            
            if r != 0:
                new_messages.append({"role": "user", "content": all_responses[i]['user'][r]})
                new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][r]})

            potential_answers = list(response_pool)
            new_message += f'Potential answers: '
            num_ans = 1
            for pa in potential_answers:
                new_message += f'{num_ans}:{pa};'
                num_ans += 1
            new_messages.append({"role": "user", "content": new_message})
            
            # other_responses = response_list[:i] + response_list[i+1:]
            # message += f'Reasoning from other agents: '
            # for k in range(len(other_responses)):
            #     message += f'Agent {k+1}: {other_responses[k]}'
            # message += 'Thought:'
            # new_messages.append({"role": "user", "content": message})
            
            response_message = client.chat.completions.create(model=model, messages=new_messages, temperature=temperature)
            response = response_message.choices[0].message.content
            all_responses[i]['user'].append(new_message)
            all_responses[i]['assistant'].append(response)

    #put all answers in a pool, and then do majority vote
    all_responses_pool = []
    for i in range(num_agents):
        answer_i = all_responses[i]['assistant'][-1]
        try:
            response_format = re.search('###(.*)###', answer_i).group(1)
            extracted = extract_answer(response_format)
            if extracted != INVALID_ANS:
                all_responses_pool.append(extracted)
        except:
            continue

    return all_responses, max(set(all_responses_pool), key=all_responses_pool.count)

In [37]:
result = []
Answer_Agent_debate(split[0], 2, 3)

({0: {'user': ['Given a question, give out your chain of thought process and the final answer.     Make sure to include your answer in the format: ###your answer###.     Give out the answer in numerical format.     Question: {\'question\': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers\' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers\' market?", \'answer\': \'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\\n#### 18\'}     Let\'s think step by step.     Answer:     ',
    'Given some potential answers given by other agents:     if you think your previous answer is wrong, and one of the potential answers is correct,     provide the correct answer and your reasoning for it;     if you think your previous answer is correct, explain why it 

In [38]:
all_answer_gpt3_text_cot_0s_agent3 = []
ori_reasoning_cot_0s = []
all_counts = -1
for s in tqdm(split[0:100]):
    all_counts += 1
    ori_question = s['question']
    
    response, ans = Answer_Agent_debate(ori_question, 2, 3)
    all_answer_gpt3_text_cot_0s_agent3.append(ans)

100%|██████████| 100/100 [14:51<00:00,  8.92s/it]


In [42]:
c, p, l = evaluation(all_answer_gpt3_text_cot_0s_agent3, golden)
c, p

(100, 1.0)